# KUchat Production Deployment

**Enterprise-Grade AI Assistant for Kasetsart University Curriculum Information**

## Overview

This notebook provides a complete production deployment of the KUchat AI assistant system, featuring:

- GPT-OSS-120B language model (120 billion parameters)
- 4-bit quantization using Unsloth optimization framework
- Retrieval-Augmented Generation (RAG) system with ChromaDB
- Web search integration (DuckDuckGo and Wikipedia)
- Public-facing Gradio web interface
- Zero local configuration required

## System Requirements

1. **Google Colab Pro+ subscription** (A100 GPU access required)
2. **Documentation repository** (automatically downloaded from GitHub)
3. **Internet connection** for model downloads and web search

## Deployment Instructions

1. Navigate to Runtime → Change runtime type → Select **A100 GPU**
2. Execute all cells sequentially from top to bottom
3. Wait for automatic documentation download from repository
4. Access the generated public URL for the demo interface

## Technical Specifications

- **Model**: GPT-OSS-120B (120B parameters)
- **Quantization**: 4-bit BitsAndBytes (Unsloth optimized)
- **Memory Footprint**: Approximately 40GB VRAM
- **Inference Speed**: 30-60 tokens per second
- **Optimization**: 2x faster inference, 75% memory reduction vs FP16
- **Quality**: Production-ready with enterprise-grade responses

---

## Step 1: Install Dependencies

In [ ]:
%%capture
# Install required packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install torch transformers accelerate bitsandbytes
!pip install langchain langchain-community chromadb sentence-transformers
!pip install duckduckgo-search wikipedia beautifulsoup4
!pip install pypdf python-multipart aiofiles
!pip install gradio

print("All packages installed successfully")

## Step 2: Import Libraries and Verify GPU

In [ ]:
import os
import torch
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
from typing import Optional, List, Tuple
import gradio as gr
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from duckduckgo_search import DDGS
import wikipedia
import time

print("=" * 60)
print("KUCHAT PRODUCTION SYSTEM - A100 GPU")
print("=" * 60)
print(f"\nGPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    if 'A100' not in torch.cuda.get_device_name(0):
        print("\nWARNING: A100 GPU not detected")
        print("Action required: Runtime → Change runtime type → A100 GPU")
else:
    print("\nERROR: No GPU detected")
    print("Please enable GPU in runtime settings")

print("\nLibraries imported successfully")

## Step 3: System Configuration

In [ ]:
# Model Configuration - Unsloth 4-bit Quantized GPT-OSS-120B
MODEL_NAME = "unsloth/gpt-oss-120b-unsloth-bnb-4bit"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Documentation folder path
DOCS_FOLDER = "./docs"

# HuggingFace authentication token (optional for this model)
HF_TOKEN = ""  # Add token from https://huggingface.co/settings/tokens if needed

print("Configuration initialized")
print(f"Model: {MODEL_NAME}")
print(f"Quantization: 4-bit BitsAndBytes (Unsloth optimized)")
print(f"Embedding Model: {EMBEDDING_MODEL}")
print(f"Documentation Path: {DOCS_FOLDER}")
print(f"Expected VRAM Usage: ~40GB (A100 compatible)")

if HF_TOKEN:
    print("HuggingFace token: Configured")
else:
    print("HuggingFace token: Not required for this model")

## Step 4: Download Documentation from Repository

Documentation files are automatically downloaded from the GitHub repository.

In [ ]:
print("Downloading documentation from GitHub repository...\n")

# Clone repository with sparse checkout (docs folder only)
!git clone --depth 1 --filter=blob:none --sparse https://github.com/themistymoon/KUchat.git
%cd KUchat
!git sparse-checkout set docs

# Move docs folder from /content/KUchat/docs to /content/docs
%cd /content
!mv KUchat/docs docs
!rm -rf KUchat

print("\nDocumentation downloaded successfully")
print(f"Location: {DOCS_FOLDER}")

# Display folder structure
!echo "\nFolder structure:"
!ls -lh docs/ | head -20

## Step 5: Load GPT-OSS-120B Language Model

In [ ]:
print("=" * 60)
print("[1/4] Loading GPT-OSS-120B Language Model")
print("=" * 60)
print("Estimated loading time: 3-5 minutes (first run)")
print("\nModel: GPT-OSS-120B (120 billion parameters)")
print("Quantization: 4-bit BitsAndBytes (Unsloth optimized)")
print("Expected VRAM usage: ~40GB\n")

start_time = time.time()

# Configure model parameters
max_seq_length = 2048  # Maximum context length (supports up to 32K)

# Load model using Unsloth's FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=None,  # Automatic dtype detection
    load_in_4bit=True,  # Enable 4-bit quantization
)

# Optimize for inference
FastLanguageModel.for_inference(model)

load_time = time.time() - start_time

print(f"\nModel loaded successfully in {load_time:.1f} seconds")
print(f"Model: GPT-OSS-120B (120B parameters)")
print(f"Quantization: 4-bit BitsAndBytes")
print(f"Maximum sequence length: {max_seq_length} tokens")
if torch.cuda.is_available():
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"Memory optimization: ~75% reduction vs FP16")

## Step 6: Initialize RAG System

In [ ]:
print("\n[2/4] Initializing Retrieval-Augmented Generation system\n")

class RAGSystem:
    def __init__(self):
        print("Loading embedding model...")
        self.embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        self.vectorstore = None
        self.docs_loaded = False
        print("Embedding model initialized")

    def load_documents(self, folder_path: str):
        if not os.path.exists(folder_path):
            print(f"WARNING: Folder {folder_path} not found")
            return

        print(f"Loading documents from {folder_path}...\n")
        documents = []
        file_count = 0
        
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.pdf'):
                    file_path = os.path.join(root, file)
                    try:
                        loader = PyPDFLoader(file_path)
                        documents.extend(loader.load())
                        file_count += 1
                        if file_count % 10 == 0:
                            print(f"  Loaded {file_count} files...")
                    except Exception as e:
                        print(f"  Error loading {file}: {e}")
                elif file.endswith(('.txt', '.md')):
                    file_path = os.path.join(root, file)
                    try:
                        loader = TextLoader(file_path)
                        documents.extend(loader.load())
                        file_count += 1
                    except Exception as e:
                        print(f"  Error loading {file}: {e}")

        if documents:
            print(f"\nSplitting {len(documents)} documents into chunks...")
            texts = self.text_splitter.split_documents(documents)
            print(f"Created {len(texts)} text chunks")
            
            print("Building vector database...")
            self.vectorstore = Chroma.from_documents(texts, self.embeddings)
            self.docs_loaded = True
            
            print(f"\nSuccessfully loaded {len(documents)} documents ({len(texts)} chunks)")
            print("RAG system operational")
        else:
            print("WARNING: No documents found in folder")

    def query(self, question: str, k: int = 3):
        if not self.docs_loaded or self.vectorstore is None:
            return None
        results = self.vectorstore.similarity_search(question, k=k)
        return "\n\n".join([doc.page_content for doc in results])

# Initialize RAG system
rag_system = RAGSystem()

<VSCode.Cell id="#VSC-8d7de5e1" language="markdown">
## Step 7: Load Curriculum Documents

In [ ]:
print("[3/4] Loading documents into RAG system...\n")
rag_system.load_documents(DOCS_FOLDER)

## Step 8: Initialize Web Search System

In [ ]:
class WebSearchSystem:
    @staticmethod
    def search_duckduckgo(query: str, max_results: int = 3):
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=max_results))
                return "\n\n".join([f"**{r['title']}**\n{r['body']}" for r in results])
        except Exception as e:
            return f"Web search failed: {str(e)}"

    @staticmethod
    def search_wikipedia(query: str):
        try:
            wikipedia.set_lang('th')
            summary = wikipedia.summary(query, sentences=3)
            return summary
        except:
            try:
                wikipedia.set_lang('en')
                summary = wikipedia.summary(query, sentences=3)
                return summary
            except Exception as e:
                return f"Wikipedia search failed: {str(e)}"

web_search = WebSearchSystem()
print("Web search system initialized")

## Step 9: Chat Function with RAG and Web Search

In [ ]:
def chat_with_bot(
    message: str,
    history: List[Tuple[str, str]],
    use_rag: bool,
    use_web_search: bool,
    temperature: float,
    max_tokens: int
) -> str:
    """
    Main chat function with RAG and web search
    """
    try:
        # Build context
        context = ""
        
        if use_rag and rag_system.docs_loaded:
            rag_context = rag_system.query(message, k=3)
            if rag_context:
                context += f"\n\nเอกสารหลักสูตรที่เกี่ยวข้อง:\n{rag_context}"
        
        if use_web_search:
            web_results = web_search.search_duckduckgo(message, max_results=2)
            context += f"\n\nข้อมูลจากอินเทอร์เน็ต:\n{web_results}"
        
        # Build conversation history
        conversation = ""
        for user_msg, bot_msg in history[-3:]:  # Last 3 exchanges
            conversation += f"คำถาม: {user_msg}\nคำตอบ: {bot_msg}\n\n"
        
        # Build prompt
        prompt = f"""คุณคือผู้ช่วยตอบคำถามเกี่ยวกับหลักสูตรมหาวิทยาลัยเกษตรศาสตร์ (Kasetsart University)
ให้ตอบคำถามอย่างละเอียด เป็นมิตร และถูกต้อง

{conversation}
คำถามใหม่: {message}
{context}

คำตอบ:"""
        
        # Generate response
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract answer (remove prompt)
        if "คำตอบ:" in response:
            answer = response.split("คำตอบ:")[-1].strip()
        else:
            answer = response[len(prompt):].strip()
        
        return answer
    
    except Exception as e:
        return f"Error: {str(e)}"

print("Chat function initialized")

## Step 10: Launch Gradio Demo Interface

In [ ]:
print("\n[4/4] Launching Gradio interface...\n")

# Create Gradio interface
with gr.Blocks(
    title="KUchat - Kasetsart University AI Assistant",
    theme=gr.themes.Soft()
) as demo:
    
    gr.Markdown("""
    # KUchat - Kasetsart University AI Assistant
    
    ผู้ช่วยตอบคำถามเกี่ยวกับหลักสูตรมหาวิทยาลัยเกษตรศาสตร์
    
    **Powered by GPT-OSS-120B (4-bit, Unsloth) on A100 GPU**
    
    ---
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=500,
                label="Chat History",
                show_copy_button=True
            )
            
            msg = gr.Textbox(
                label="Your Question",
                placeholder="ถามคำถามเกี่ยวกับหลักสูตร เช่น 'หลักสูตรวิศวกรรมคอมพิวเตอร์มีอะไรบ้าง'",
                lines=2
            )
            
            with gr.Row():
                submit_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear")
        
        with gr.Column(scale=1):
            gr.Markdown("### Settings")
            
            use_rag = gr.Checkbox(
                label="Use RAG (Curriculum Documents)",
                value=True,
                info="Search in curriculum documents"
            )
            
            use_web_search = gr.Checkbox(
                label="Use Web Search",
                value=False,
                info="Search online for latest information"
            )
            
            temperature = gr.Slider(
                minimum=0.1,
                maximum=1.0,
                value=0.7,
                step=0.1,
                label="Temperature",
                info="Higher values increase creativity"
            )
            
            max_tokens = gr.Slider(
                minimum=128,
                maximum=2048,
                value=512,
                step=128,
                label="Max Tokens",
                info="Maximum response length"
            )
            
            gr.Markdown("""
            ---
            ### System Information
            - **Model**: GPT-OSS-120B
            - **Parameters**: 120B (4-bit)
            - **GPU**: A100 80GB
            - **VRAM Usage**: ~40GB
            - **Inference Speed**: 30-60 tokens/sec
            - **Optimization**: Unsloth Framework
            - **RAG Database**: ChromaDB
            - **Documents**: 170+ curricula
            """)
    
    gr.Markdown("""
    ---
    ### Example Questions:
    - หลักสูตรวิศวกรรมคอมพิวเตอร์มีอะไรบ้าง
    - คณะวิทยาศาสตร์มีกี่สาขา
    - ค่าเทอมคณะบริหารธุรกิจเท่าไหร่
    - วิทยาการคอมพิวเตอร์ต่างจากวิศวกรรมคอมพิวเตอร์อย่างไร
    """)
    
    # Chat function
    def respond(message, chat_history, use_rag, use_web_search, temperature, max_tokens):
        bot_message = chat_with_bot(
            message, chat_history, use_rag, use_web_search, temperature, max_tokens
        )
        chat_history.append((message, bot_message))
        return "", chat_history
    
    # Event handlers
    submit_btn.click(
        respond,
        inputs=[msg, chatbot, use_rag, use_web_search, temperature, max_tokens],
        outputs=[msg, chatbot]
    )
    
    msg.submit(
        respond,
        inputs=[msg, chatbot, use_rag, use_web_search, temperature, max_tokens],
        outputs=[msg, chatbot]
    )
    
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# Launch with public URL
print("="*60)
print("LAUNCHING KUCHAT DEMO")
print("="*60)
print("\nStarting Gradio interface...\n")

demo.launch(
    share=True,  # Creates public URL
    debug=False,
    show_error=True,
    server_port=7860
)

print("\n" + "="*60)
print("KUCHAT IS LIVE")
print("="*60)
print("\nPublic URL generated above")
print("Share the URL with anyone to use the chatbot")
print("Keep this cell running to keep the demo alive")
print("="*60)

---

## Demo Features

### Chat Interface
- Gradio UI with chat history
- Real-time responses from GPT-OSS-120B
- Copy/paste support

### Controls
- **RAG Toggle**: Search curriculum documents
- **Web Search**: Get latest online information
- **Temperature**: Adjust creativity (0.1-1.0)
- **Max Tokens**: Control response length

### Public Access
- Share URL works for 72 hours
- Anyone can access without login
- Multiple users can chat simultaneously

### Performance
- **Model**: GPT-OSS-120B (120B parameters)
- **Quantization**: 4-bit BNB (Unsloth optimized)
- **GPU**: A100 80GB (~40GB VRAM used)
- **Speed**: 30-60 tokens/second (2x faster)
- **Quality**: Production-ready
- **VRAM Savings**: 75% compared to FP16

---

## Troubleshooting

### No A100 GPU?
- Go to: Runtime → Change runtime type → A100 GPU
- Requires Google Colab Pro+ subscription

### Model loading fails?
- Check HuggingFace token is set
- Verify internet connection
- Try restarting runtime

### Demo stops working?
- Cell must keep running for demo to work
- Colab disconnects after ~12 hours idle
- Re-run cell 10 to restart demo

---

## Cost Estimate

**Google Colab Pro+**: ~$50/month
- A100 GPU access
- ~$1-2 per hour of usage
- Background execution
- Priority access

**Alternative**: Use test version (T4 GPU) for free
- See `colab_backend_test.ipynb`
- Smaller model but still functional
- Good for testing/development

---